In [ ]:
import pandas as pd
import yaml
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

In [ ]:
config

In [ ]:
df = pd.read_csv(config ["data"]["raw"]["file_raw"])
df

In [ ]:
df.isna().sum()

In [ ]:
average_bmi = df['bmi'].mean()

df["bmi"] = df["bmi"].fillna(average_bmi)
df

In [ ]:
df.isna().sum()

In [ ]:
df = df.drop("id", axis=1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)

ohe.fit(df[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
df_ohe = ohe.transform(df[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
df_ohe

In [ ]:
df_ohe = pd.DataFrame(df_ohe, columns=ohe.get_feature_names_out(), index=df.index)
df_ohe

In [ ]:
df_num = df.select_dtypes(include=['number'])

df_fin = pd.concat([df_num, df_ohe], axis = 1)

df_model = df_num.drop("stroke", axis=1)

target = df.stroke


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np

sns.boxplot(x = 'stroke', y = 'age', data = df_fin)
plt.show()

In [ ]:
sns.boxplot(x = 'stroke', y = 'avg_glucose_level', data = df_fin)
plt.show()

In [ ]:
sns.boxplot(x = 'stroke', y = 'bmi', data = df_fin)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming df_fin is your existing DataFrame
corr = np.abs(df_fin.corr())

mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with adjusted annotation font size and mask
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=1,
    square=True,
    linewidths=.5,
    cbar_kws={"shrink": .5},
    annot=True,
    annot_kws={"size": 8},  # Adjust the font size for annotation values
    fmt=".2f"  # Control the format of the annotation values
)

# Customize axis labels for better readability
plt.xticks(fontsize=10, rotation=45, ha="right")  # Adjust x-axis labels
plt.yticks(fontsize=10)  # Adjust y-axis labels

plt.show()

In [ ]:
df_num = df.select_dtypes(include=['number'])
df_num = df_num.drop("id", axis=1)
df_num

In [ ]:
df.dtypes